# How to FineTune Llama 3 with  SFTTrainer 
Hello everyone, today we are going to show how we can Fine Tune Llama 3 with SFTTrainer 
First we are going to perform a simmple Fine Tunning by using SFTTrainer

## Step 1 - Installation of Pytorch
The first step is install pythorch v 2.2.1 with Cuda 12.1 

In [1]:
!python -m pip install --upgrade pip
!pip3 install torch==2.2.1 torchvision torchaudio   xformers --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


## Step 3 - Installation of Uslotch packages

In [2]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft \
    accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps  trl peft accelerate bitsandbytes
!pip install datasets
!pip install hyperopt
!pip install optuna    
pass

  Cloning https://github.com/unslothai/unsloth.git to /tmp/wsuser/pip-install-fa66fper/unsloth_294960f3f736405cbc5c9ef0cfe45de6
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/wsuser/pip-install-fa66fper/unsloth_294960f3f736405cbc5c9ef0cfe45de6
  Resolved https://github.com/unslothai/unsloth.git to commit d4512f7c138a254d789fcba247b9c363a8aa2e25
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


## Step 4 - Analysis of our infrastructure
In ordering to perform any training it is important to know our system in order to take the full advantage of the system.

In [3]:
!python -m xformers.info
!python -m bitsandbytes
!nvidia-smi

Unable to find python bindings at /usr/local/dcgm/bindings/python3. No data will be captured.
xFormers 0.0.25
memory_efficient_attention.ckF:                    unavailable
memory_efficient_attention.ckB:                    unavailable
memory_efficient_attention.ck_decoderF:            unavailable
memory_efficient_attention.ck_splitKF:             unavailable
memory_efficient_attention.cutlassF:               available
memory_efficient_attention.cutlassB:               available
memory_efficient_attention.decoderF:               available
memory_efficient_attention.flshattF@v2.5.6:        available
memory_efficient_attention.flshattB@v2.5.6:        available
memory_efficient_attention.smallkF:                available
memory_efficient_attention.smallkB:                available
memory_efficient_attention.triton_splitKF:         unavailable
indexing.scaled_index_addF:                        unavailable
indexing.scaled_index_addB:                        unavailable
indexing.index_select:

## Step 5 Login to Hugging Face

In [1]:
token=""
from huggingface_hub import login, logout
login(token) # non-blocking login

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/wsuser/.cache/huggingface/token
Login successful


## Step 5  Simple Fine Tunning Method

First let us show the simplest method that is given by  SFTTrainer

In [1]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import LoraConfig
from trl import SFTTrainer
from transformers import TrainingArguments
# Load the dataset
dataset_name = "ruslanmv/ai-medical-dataset"
dataset = load_dataset(dataset_name, split="train")
# Select the first 1000 rows of the dataset
dataset = dataset.select(range(100))
# Device map
device_map = 'auto'  # for PP and running with `python test_sft.py`
# Load the model + tokenizer
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_cache=False,
    device_map=device_map
)
# PEFT config
lora_alpha = 16
lora_dropout = 0.1
lora_r = 32  # 64
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["k_proj", "q_proj", "v_proj", "up_proj", "down_proj", "gate_proj"],
    modules_to_save=["embed_tokens", "input_layernorm", "post_attention_layernorm", "norm"],
)
# Args
max_seq_length = 512
output_dir = "./results"
per_device_train_batch_size = 2  # reduced batch size to avoid OOM
gradient_accumulation_steps = 4#2
optim = "adamw_torch"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4#2e-4
max_grad_norm = 0.3
max_steps = 5  
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,  # gradient checkpointing
    #report_to="wandb",
)
# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="context",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Memory statistics before training
gpu_statistics = torch.cuda.get_device_properties(0)
reserved_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
max_memory = round(gpu_statistics.total_memory / 1024**3, 2)
print(f"Reserved Memory: {reserved_memory}GB")
print(f"Max Memory: {max_memory}GB")


# Train :)
trainer.train()

# Memory statistics after training
used_memory = round(torch.cuda.max_memory_allocated() / 1024**3, 2)
used_memory_lora = round(used_memory - reserved_memory, 2)
used_memory_persentage = round((used_memory / max_memory) * 100, 2)
used_memory_lora_persentage = round((used_memory_lora / max_memory) * 100, 2)
print(f"Used Memory: {used_memory}GB ({used_memory_persentage}%)")
print(f"Used Memory for training(fine-tuning) LoRA: {used_memory_lora}GB ({used_memory_lora_persentage}%)")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


Reserved Memory: 6.91GB
Max Memory: 15.77GB


/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.256300
2,2.422400
3,3.420900
4,2.049800
5,2.573800


Used Memory: 8.03GB (50.92%)
Used Memory for training(fine-tuning) LoRA: 1.12GB (7.1%)


| Training Loss Range | Possible Meaning |
| --- | --- |
| **< 0.1** | **Excellent** learning: The model is fitting the training data very well and is likely to generalize well to new data. |
| **0.1 - 0.5** | **Good** learning: The model is learning from the data, but may not be optimal. Further tuning of hyperparameters may be necessary. |
| **0.5 - 1.0** | **Fair** learning: The modes learning, but the loss is relatively high. The model may not be capturing the underlying patterns in the data. |
| **1.0 - 2.0** | **Poor** learning: The model is not learning effectively. The loss is high, and the model may be overfitting or underfitting the data. |
| **> 2.0** | **No** learning: The model is not learning from the data. The loss is very high, and the model may be stuck in a local minimum or has convergence issues. |


In [2]:
## Save and push the adapter to HF
import os
current_directory = os.getcwd()
# New model name
new_model="ai-medical-model" #"Medical-Mind-Llama-3-8b"
# Save the fine-tuned model
save_path = os.path.join(current_directory, "models", new_model)
os.makedirs(save_path, exist_ok=True)  

In [3]:
# Save model local
#trainer.save_model("ai-medical-model")
#tokenizer.save_pretrained("ai-medical-model")

In [5]:
#trainer.model.save_pretrained(save_path)

trainer.save_model(save_path)
* Saves only the model's weights to the specified `save_path`.
* Does not save the optimizer state, scheduler state, or any other training-related information.
* Saves the model in a format that can be loaded using `torch.load()` or similar function
* Typically used when you only need to save the model's weights and don't need to resume training from the saved checkpoint.

In [4]:
#This ONLY saves the LoRA adapters, and not the full model. 
trainer.save_model(save_path)

/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


To save the complete model, including the LoRA weights, you can use  method from the `trl` library.
The command to save the complete model is:

In [ ]:
#This save Model. 

In [5]:
model.save_pretrained(save_path)

In [6]:
#This save tokenizer. 
tokenizer.save_pretrained(save_path)

('/home/wsuser/work/models/ai-medical-model/tokenizer_config.json',
 '/home/wsuser/work/models/ai-medical-model/special_tokens_map.json',
 '/home/wsuser/work/models/ai-medical-model/tokenizer.json')

In [7]:
!ls /home/wsuser/work/models/ai-medical-model/

adapter_config.json		  model.safetensors.index.json
adapter_model.safetensors	  README.md
config.json			  special_tokens_map.json
generation_config.json		  tokenizer_config.json
model-00001-of-00003.safetensors  tokenizer.json
model-00002-of-00003.safetensors  training_args.bin
model-00003-of-00003.safetensors


In [8]:
save_path

'/home/wsuser/work/models/ai-medical-model'

In [9]:
# Get the list of files in the directory
files_in_model_dir = os.listdir(save_path)
# Print the list of files
print("Files in the directory:")
for file in files_in_model_dir:
    print(file)

Files in the directory:
generation_config.json
model-00002-of-00003.safetensors
model-00003-of-00003.safetensors
special_tokens_map.json
README.md
model.safetensors.index.json
tokenizer_config.json
adapter_model.safetensors
adapter_config.json
model-00001-of-00003.safetensors
tokenizer.json
config.json
training_args.bin


In [10]:
test_inference=True
if test_inference:
    ## Inference 
    import torch
    from transformers import AutoModelForCausalLM, AutoTokenizer
    max_seq_length = 2048
    dtype = None
    load_in_4bit = True
    fine_tuned_model = AutoModelForCausalLM.from_pretrained(save_path, load_in_4bit=load_in_4bit)
    tokenizer = AutoTokenizer.from_pretrained(save_path)

    # Prepare the model for inference
    fine_tuned_model.eval()
if test_inference:
    question="This is the question: What was the main cause of the inflammatory CD4+ T cells?"
    prompt = f"<|start_header_id|>system<|end_header_id|> You are a Medical AI chatbot assistant. <|eot_id|><|start_header_id|>User: <|end_header_id|>{question}<|eot_id|>"
    # Tokenizing the input and generating the output
    #prompt = f"{question}"
    # Tokenizing the input and generating the output
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = fine_tuned_model.generate(**inputs, max_new_tokens=256, use_cache=True)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    # Try Remove the prompt
    try:
        # Split the answer at the first line break, assuming system intro and question are on separate lines
        answer_parts = answer.split("\n", 1)
        # If there are multiple parts, consider the second part as the answer
        if len(answer_parts) > 1:
          answers = answer_parts[1].strip()  # Remove leading/trailing whitespaces
        else:
          answers = ""  # If no split possible, set answer to empty string
        print(f"Answer: {answers}")   
    except:
        print(answer)

/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Answer: The main cause of the inflammatory CD4+ T cells is still a topic of ongoing research and debate in the scientific community. However, several factors have been identified as contributing to the development of inflammatory CD4+ T cells:

1. **Viral infections**: Viruses such as HIV, HTLV-1, and Epstein-Barr virus can trigger the activation of CD4+ T cells, leading to inflammation.
2. **Autoimmune disorders**: Conditions like rheumatoid arthritis, lupus, and multiple sclerosis can cause CD4+ T cells to become activated and produce pro-inflammatory cytokines.
3. **Allergic reactions**: Exposure to allergens, such as pollen, dust, or certain foods, can trigger the activation of CD4+ T cells and lead to inflammation.
4. **Bacterial infections**: Bacterial infections, such as tuberculosis, can activate CD4+ T cells and lead to inflammation.
5. **Environmental toxins**: Exposure to environmental toxins, such as pesticides or heavy metals, has been linked to the development of inflamma

In [13]:
import os
from huggingface_hub import HfApi
def upload_folder(folder_path, repository_name, path_in_repo):
    api = HfApi()
    
    # Check if the repository exists, if not, create it
    repo_exists = api.repo_exists(repository_name)
    if not repo_exists:
        api.create_repo(repository_name)
        print(f"Repository '{repository_name}' created on Hugging Face Hub.")

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(file_path, folder_path)
            repo_path = os.path.join(path_in_repo, relative_path)
            api.upload_file(path_or_fileobj=file_path, repo_id=repository_name, path_in_repo=repo_path)
            print(f"{repo_path} uploaded to {repository_name}")

In [14]:
# Define the repository name and path in the repository
repository_name = "ruslanmv/"+new_model
path_in_repo = ""

In [15]:
repository_name

'ruslanmv/ai-medical-model'

In [20]:
# Push the model to an organization with the name "my-finetuned".
#model.push_to_hub(repository_name)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ruslanmv/ai-medical-model/commit/cd12446e821caa8e46509f2d65f70bd4307535d8', commit_message='Upload LlamaForCausalLM', commit_description='', oid='cd12446e821caa8e46509f2d65f70bd4307535d8', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
# Push the tokenizer to an organization with the name 
#tokenizer.push_to_hub(repository_name)

In [17]:
repository_name

'ruslanmv/ai-medical-model'

In [18]:
# Upload the folder and its contents to the repository
upload_folder(save_path, repository_name, path_in_repo)

Repository 'ruslanmv/ai-medical-model' created on Hugging Face Hub.
generation_config.json uploaded to ruslanmv/ai-medical-model


model-00002-of-00003.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model-00002-of-00003.safetensors uploaded to ruslanmv/ai-medical-model


model-00003-of-00003.safetensors:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

model-00003-of-00003.safetensors uploaded to ruslanmv/ai-medical-model
special_tokens_map.json uploaded to ruslanmv/ai-medical-model
README.md uploaded to ruslanmv/ai-medical-model
model.safetensors.index.json uploaded to ruslanmv/ai-medical-model
tokenizer_config.json uploaded to ruslanmv/ai-medical-model


...

adapter_model.safetensors uploaded to ruslanmv/ai-medical-model
adapter_config.json uploaded to ruslanmv/ai-medical-model


model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors uploaded to ruslanmv/ai-medical-model
tokenizer.json uploaded to ruslanmv/ai-medical-model
config.json uploaded to ruslanmv/ai-medical-model


training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

training_args.bin uploaded to ruslanmv/ai-medical-model


In [1]:
test_inference_from_repo=True
if test_inference_from_repo:
    ## Inference 
    import torch
    from transformers import AutoModelForCausalLM, AutoTokenizer
    max_seq_length = 2048
    dtype = None
    load_in_4bit = True
    repository_name = repository_name if 'repository_name' in locals() else 'ruslanmv/ai-medical-model'
    fine_tuned_model = AutoModelForCausalLM.from_pretrained(repository_name, load_in_4bit=load_in_4bit)
    tokenizer = AutoTokenizer.from_pretrained(repository_name)    
    # Prepare the model for inference
    fine_tuned_model.eval()

/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
if test_inference_from_repo:
    question="This is the question: What was the main cause of the inflammatory CD4+ T cells?"
    prompt = f"<|start_header_id|>system<|end_header_id|> You are a Medical AI chatbot assistant. <|eot_id|><|start_header_id|>User: <|end_header_id|>{question}<|eot_id|>"
    # Tokenizing the input and generating the output
    #prompt = f"{question}"
    # Tokenizing the input and generating the output
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = fine_tuned_model.generate(**inputs, max_new_tokens=256, use_cache=True)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    # Try Remove the prompt
    try:
        # Split the answer at the first line break, assuming system intro and question are on separate lines
        answer_parts = answer.split("\n", 1)
        # If there are multiple parts, consider the second part as the answer
        if len(answer_parts) > 1:
          answers = answer_parts[1].strip()  # Remove leading/trailing whitespaces
        else:
          answers = ""  # If no split possible, set answer to empty string
        print(f"Answer: {answers}")   
    except:
        print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Answer: The main cause of inflammatory CD4+ T cells is often attributed to the activation of CD4+ T cells, which is a type of T lymphocyte that plays a central role in the immune system. CD4+ T cells become activated when they recognize and bind to antigens presented by antigen-presenting cells (APCs), such as dendritic cells and macrophages.

This activation triggers a cascade of signaling pathways that lead to the production of pro-inflammatory cytokines, such as interleukin-2 (IL-2), tumor necrosis factor-alpha (TNF-alpha), and interferon-gamma (IFN-gamma). These cytokines promote the recruitment and activation of other immune cells, including neutrophils, macrophages, and other T cells, which can lead to an inflammatory response.

In addition to antigen presentation, other factors can contribute to the activation of CD4+ T cells, including:

1. Cytokine signals: Cytokines such as IL-12, IL-23, and IL-1 can stimulate CD4+ T cells and promote their activation.
2. Co-stimulatory molec

In [4]:
if test_inference_from_repo:
    question="This is the question: Whet does Scoliosis refers to."
    prompt = f"<|start_header_id|>system<|end_header_id|> You are a Medical AI chatbot assistant. <|eot_id|><|start_header_id|>User: <|end_header_id|>{question}<|eot_id|>"
    # Tokenizing the input and generating the output
    #prompt = f"{question}"
    # Tokenizing the input and generating the output
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = fine_tuned_model.generate(**inputs, max_new_tokens=256, use_cache=True)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    # Try Remove the prompt
    try:
        # Split the answer at the first line break, assuming system intro and question are on separate lines
        answer_parts = answer.split("\n", 1)
        # If there are multiple parts, consider the second part as the answer
        if len(answer_parts) > 1:
          answers = answer_parts[1].strip()  # Remove leading/trailing whitespaces
        else:
          answers = ""  # If no split possible, set answer to empty string
        print(f"Answer: {answers}")   
    except:
        print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: Scoliosis refers to a medical condition where the spine curves abnormally to one side or the other. This curvature can occur in the thoracic (chest) or lumbar (lower back) regions of the spine, or in both. Scoliosis can be congenital (present at birth) or acquired (develop over time) and can affect individuals of all ages.

The condition can cause a range of symptoms, including:

1. Back pain or stiffness
2. Difficulty breathing (in severe cases)
3. Rib hump or uneven shoulders
4. Uneven hips or shoulders
5. Difficulty standing up straight

Scoliosis can be classified into several types, including:

1. Infantile scoliosis: Occurs in children under the age of 5
2. Juvenile scoliosis: Occurs in children between the ages of 5 and 10
3. Adolescent idiopathic scoliosis: Occurs in teenagers and young adults, often without a known cause
4. Adult scoliosis: Occurs in adults, often due to degenerative conditions such as osteoporosis or spinal injuries

Treatment for scoliosis depends on